### TECHIN509_Melody_Generation

In [1]:
import random
from collections import defaultdict, Counter
from typing import List, Dict, Tuple


# -----------------------------
# Part 1 — Overall structure
# -----------------------------
# 1) Represent each melody as a list of "notes" (strings).
# 2) Build a bigram table: current_note -> Counter(next_note -> count).
# 3) Generate new melodies by walking the bigram model.
# 4) Add start (^) and end ($) tokens so melodies can stop naturally.
# 5) Provide helper functions for printing and analysis.


# -----------------------------
# Part 2 — Build the bigram model
# -----------------------------

def build_bigram_counts(
    melodies: List[List[str]],
    use_boundaries: bool = False,
    start_token: str = "^",
    end_token: str = "$",
) -> Dict[str, Counter]:
    """
    Build a bigram table from a list of melodies.
    Each melody is a list of note tokens (e.g., ["E4", "F#4", "G4", ...]).

    If use_boundaries is True, we prepend ^ and append $ to each melody,
    so the model can learn when to start and stop.
    """
    model: Dict[str, Counter] = defaultdict(Counter)

    for melody in melodies:
        if not melody:
            continue

        if use_boundaries:
            tokens = [start_token] + melody + [end_token]
        else:
            tokens = melody

        # zip over bigrams: (current_note, next_note)
        for cur, nxt in zip(tokens, tokens[1:]):
            model[cur][nxt] += 1

    return model


def pretty_print_bigram(model: Dict[str, Counter]) -> None:
    """Print the bigram model in a readable way."""
    for cur, counter in model.items():
        print(f"{cur} → {dict(counter)}")


# -----------------------------
# Part 3 — Generate new melodies
# -----------------------------

def weighted_choice(counter: Counter) -> str:
    """
    Choose one key from a Counter, with probability proportional to its count.
    """
    notes = list(counter.keys())
    weights = list(counter.values())
    return random.choices(notes, weights=weights, k=1)[0]


def generate_melody(
    model: Dict[str, Counter],
    max_length: int = 16,
    start_token: str = "^",
    end_token: str = "$",
    forbid_triple: bool = True,
) -> List[str]:
    """
    Generate one melody from a bigram model with start/end tokens.

    - Starts from `start_token`
    - At each step, picks a next note based on the bigram counts.
    - Stops if it hits `end_token`, or when max_length is reached.
    - If forbid_triple is True, tries to avoid repeating the same note 3+ times.
    """
    current = start_token
    melody: List[str] = []

    for _ in range(max_length):
        if current not in model:
            break

        next_counter = model[current]
        if not next_counter:  # no outgoing transitions
            break

        next_note = weighted_choice(next_counter)

        # Stop if we reached the end token
        if next_note == end_token:
            break

        # Optional constraint: avoid 3 identical notes in a row
        if forbid_triple and len(melody) >= 2:
            attempts = 0
            while (
                next_note == melody[-1] == melody[-2]
                and attempts < 5
            ):
                # Try re-sampling a different note
                next_note = weighted_choice(next_counter)
                attempts += 1

        melody.append(next_note)
        current = next_note

    return melody


# -----------------------------
# Part 5 — Analysis helpers
# -----------------------------

def most_common_transition(model: Dict[str, Counter]) -> Tuple[str, str, int]:
    """
    Find the most common bigram transition in the model.
    Returns (from_note, to_note, count).
    """
    best_from = best_to = None
    best_count = 0

    for cur, counter in model.items():
        if not counter:
            continue
        to, count = counter.most_common(1)[0]
        if count > best_count:
            best_from, best_to, best_count = cur, to, count

    return best_from, best_to, best_count


# -----------------------------
# Example dataset (Part 2)
# -----------------------------
# Example 1: notes without duration
melody1 = "E4 F#4 G4 A4 B4 A4 G4 F#4 E4".split()

# Example 2: notes with duration encoded in the token
melody2 = (
    "F4_1.3 A4_0.45 F4_0.45 A4_0.45 "
    "A#4_1.34 D5_0.45 A#4_0.45 D5_0.45 F5_2.23"
).split()

dataset = [melody1, melody2]


# -----------------------------
# Putting everything together
# -----------------------------

if __name__ == "__main__":
    random.seed(509)  # for reproducibility

    # Build bigram model with start/end tokens
    bigram_model = build_bigram_counts(
        dataset,
        use_boundaries=True,
        start_token="^",
        end_token="$",
    )

    print("Bigram model:")
    pretty_print_bigram(bigram_model)
    print()

    # Generate and print at least 3 sample melodies
    print("Generated melodies:")
    for i in range(1, 4):
        melody = generate_melody(
            bigram_model,
            max_length=12,
            start_token="^",
            end_token="$",
            forbid_triple=True,
        )
        print(f"{i}.", " ".join(melody))
    print()

    # Show most common transition
    frm, to, cnt = most_common_transition(bigram_model)
    print(f"Most common transition: {frm} → {to} (count = {cnt})")


Bigram model:
^ → {'E4': 1, 'F4_1.3': 1}
E4 → {'F#4': 1, '$': 1}
F#4 → {'G4': 1, 'E4': 1}
G4 → {'A4': 1, 'F#4': 1}
A4 → {'B4': 1, 'G4': 1}
B4 → {'A4': 1}
F4_1.3 → {'A4_0.45': 1}
A4_0.45 → {'F4_0.45': 1, 'A#4_1.34': 1}
F4_0.45 → {'A4_0.45': 1}
A#4_1.34 → {'D5_0.45': 1}
D5_0.45 → {'A#4_0.45': 1, 'F5_2.23': 1}
A#4_0.45 → {'D5_0.45': 1}
F5_2.23 → {'$': 1}

Generated melodies:
1. F4_1.3 A4_0.45 F4_0.45 A4_0.45 F4_0.45 A4_0.45 A#4_1.34 D5_0.45 F5_2.23
2. E4
3. F4_1.3 A4_0.45 F4_0.45 A4_0.45 F4_0.45 A4_0.45 F4_0.45 A4_0.45 F4_0.45 A4_0.45 F4_0.45 A4_0.45

Most common transition: ^ → E4 (count = 1)
